<a href="https://colab.research.google.com/github/svetaepc/ds_py/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Char-based text generation with LSTM

In [ ]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

## 1. Готовим данные для сети
Токенизируем текст. Превращаем его в индексы. Т.е. достаем все уникальные символы: буквы, пробелы, знаки препинания в тексте и каждому символу присваиваем число.
Это будем называть словарем. А числа — индексами.

Делаем прямой и обратный (индексы в символы) словарь. А потом проходимся по всему тексту и превращаем с помощью составленного нами словаря каждый символ в индекс.

Привет, как дел
ривет, как дела

In [ ]:
TRAIN_TEXT_FILE_PATH = 'train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])

    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', '\n', 'о', 'а', 'е', 'т', 'и', 'н', 'л', 'с', 'р', 'в', 'к', 'у', 'д', 'м', 'п', 'ь', 'я', 'ы', '.', 'й', 'г', 'б', 'з', ',', 'ч', 'ж', 'х', 'ш', 'ю', '!', 'П', 'С', 'В', '…', 'Н', '—', 'ц', 'К', '?', 'О', 'Т', 'М', 'Д', 'Г', 'И', 'З', 'Б', 'Л', 'А', 'щ', '-', ':', '»', '«', 'У', 'Р', 'Ч', 'Х', 'э', 'Я', 'Ж', 'ф', 'Е', '*', 'Ш', 'Ф', 'Э', 'ё', '–', 'Ц', 'e', '0', 'o', 'ъ', 't', 'a', '(', 'n', 'k', ')', 'C', 'X', 'r', '"', 'Й', 'c', 'Щ', "'", 'p', 'm', 'Ь', 's', 'i', '9', '1', 'w', 'B', 'M', 'd', '2', 'Ю', 'N', 'b', 'u', 'D', 'l', 'K', 'Ё', 'V', 'R', 'h', 'U', 'I', '<', '8', '4', '#', 'H', '6', '7']


## 2. Генерируем батчи из текста
Генерируем из последовательности наших индексов батчи (сразу несколько строк текста) для обучения сети. Не будем усложнять, просто достанем несколько случайных строк из текста фиксированной длины.

Будем генерировать сразу обучающую выборку (то, на чем будем учить сеть) и таргет для нее. Таргет (правильные ответы для нейросети) — это просто сдвинутый на один символ вперед текст.

In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16


# `get_batch()`, которая возвращает пару тензоров `torch.stack(trains, dim=0)` и `torch.stack(targets, dim=0)`.
def get_batch(sequence):
    """Переменная `sequence` является последовательностью элементов,
    которые должны быть разбиты на куски длиной `SEQ_LEN`.
    Переменная `BATCH_SIZE` определяет число кусков, которые должны быть сгенерированы.
    Каждый кусок включает в себя `SEQ_LEN` элементов.
    """
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        """Затем в цикле `for` генерируются `BATCH_SIZE` кусков, для каждого из
        которых сначала генерируется случайный начальный индекс `batch_start`
        от `0` до `len(sequence) - SEQ_LEN` и берется подпоследовательность
        `chunk` длиной `SEQ_LEN` начиная с `batch_start`
        """
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        """Далее, из данной подпоследовательности, создается тензор `train`
        из элементов `chunk[:-1]` (используется срез от начала до последнего элемента),
        и тензор `target` из элементов `chunk[1:]`
         (используется срез от второго элемента до конца) """
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
        """возвращает пару тензоров `torch.stack(trains, dim=0)`
        и `torch.stack(targets, dim=0)`, которые являются стеком тензоров `train`
         и `target` по первому измерению (ось 0), т.е. получается тензор размера
         `(BATCH_SIZE, SEQ_LEN-1, 1)` для `train` и `(BATCH_SIZE, SEQ_LEN-1, 1)` для `target`"""
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

## 3. Пишем функцию, которая генерирует текст
Теперь напишем функцию, которая предсказывает текст с помощью нашей обученной нейросети. Это удобно сделать заранее, чтобы смотреть, что генерирует сеть во время обучения.

Сеть предсказывает нам вероятности следующей буквы, и мы с помощью этих вероятностей достаем случайно по одной букве. Если повторить операцию 1000 раз, получим текст из 1000 символов.

Параметр start_text нам нужен, чтобы было что-то, для чего предсказывать следующий символ. У нас этот символ по умолчанию — пробел, и задача сети сначала — предсказать следующий символ после пробела. Потом — следующий после этих 2-х символов. И т.д.

Параметр temp — это уровень «случайности» генерируемого текста. Так называемая «температура» с отсылкой к понятию «энтропии». То, на что делим логиты в softmax. Поставим высокую — вероятность каждой буквы будет почти одинакова и текст превратится в случайную белиберду. Поставим низкую — каждый раз будем предсказывать одно и то же и можем зациклиться на одной фразе.

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char

    return predicted_text

## 4. Создаем класс нашей нейросети

1. Превращаем каждый символ на входе сети в вектор (так называемный эмбеддинг).
2. Скармливаем эти векторы нашему LSTM слою. У этого слоя есть особенность: он работает не независимо для каждого символа, а помнит, что к нему раньше приходило на вход. Притом, помнит не все: ненужное он умеет забывать. Такие слои называют рекуррентными и часто используют при работе с последовательностями.
3. Выходы из LSTM слоя пропускаем через Dropout. Этот слой «мешает» сети учиться, чтобы ей сложнее было выучить весть текст.
4. Дальше отправляем выход из Dropout на линейный слой размерности словаря, чтобы на выходе получить столько чисел, сколько у нас символов в словаре. Потом мы этот вектор чисел будем превращать в «вероятности» каждого символа с помощью функции softmax.

In [ ]:
class TextRNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

## 5. Создаем нейросеть и обучаем ее
Теперь создаем нейросеть и обучаем ее. LSTM блок принимает немного другой формат батча:

[SEQ_LEN x BATCH_SIZE x 1], поэтому делаем permute для тензоров train и target, чтобы поменять 0 и 1 размерность местами.

Параметры нейросети, которые может понадобиться подкрутить:

- hidden_size — влияет на сложность сети. Стоит повышать для текстов большого размера. Если выставить большое значение для текста маленького размера, то сеть просто выучит весь текст и будет генерировать его же.
- n_layers — опять же, влияет на сложность сети. Грубо говоря, позволяет делать несколько LSTM слоев подряд просто меняя эту цифру.
- embedding_size — размер обучаемого эмбеддинга. Можно выставить в несколько раз меньше размера словаря (числа уникальных символов в тексте) или примерно такой же. Больше — нет смысла.
Дальше — стандартный для PyTorch цикл обучения нейросети: выбираем функцию потерь, оптимизатор и настраиваем расписание, по которому меняем шаг оптимизатора. В нашем случае снижаем шаг в 2 раза, если ошибка (loss) не падает 5 шагов подряд.

In [ ]:
%%time
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 100000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Выходные данные были обрезаны до нескольких последних строк (5000).
 
 уж не осталось
 не замерзнут ли?
 
 Вот скоро уходит
 Станик в темноте —
 что улыбнетс
Loss: 0.6367264187335968
 —
 была начнется сорный бола
 Стоит ли любить?
 
 Совсем не Байрон.
 и слог в ладонец. Давору кувший
 Друг все ближе на луну.
 Столько хлопотом
 
 света. Путь на светвет
 Под сакурою
 
 Порножурналы
 
Loss: 0.6384252464771271
 —
 поздняя осень
 
 Порножурналы
 В положный помню,
 Пойти напиться?
 
 Старый ветерок?
 Вот тебе и май…
 
 Лето на дворе…
 Перва саморгалось карто,
 Если под снегом.
 
 Стоит ли любить?
 Стоит ли люб
Loss: 0.638322058916092
 —
 весна на дворе…
 
 Осень на дворе…
 Партии моей парус
 
 Поймали вода.
 Хотел на мог ябина
 Милая в саду лесной
 Как отрезало глячь насущив
 не то что такая вихон
 Проснулся в поле.
 
 Старый крика
Loss: 0.6401946234703064
 Сегуна спасил
 но не был корей!
 
 полно простое —
 По склону холма. быстрей
 pолзи, улитка!
 
 Весенний дождь.
 Вот приходит не спросил

Рекомендую поиграть с параметром temp и, конечно же, start_text. С помощью start_text можно попробовать «задать тему/направление» для генерируемого текста.

In [ ]:
model.eval()

print(evaluate(
    model,
    char_to_idx,
    idx_to_char,
    temp=0.45,
    prediction_len=1000,
    start_text='Луна'
    )
)

Лунахвуй тихо микан,
 Мой год утро…
 
 Может и в ухо…
 Вот скоро уходит
 
 Любили его!
 А он кустик последний глюки
 Терзанья духа
 
 любви все раздолба
 вы обвежь! Диход
 
 Нет места смерти
 прохладно веселый
 
 Как дальше жить мне?
 Тоже все часто.
 
 Старый правтра
 
 От поцелуев…
 Купит на ветках
 И не помню, не смыслышки
 и утии мороз.
 
 Уж стоит ли любить?
 Нет места смерти
 
 Отдай водку промешкал
 примчался верхом в Москву
 Не подарит Дубинин
 Голос цикады
 
 паутину, мох
 А в туалетется в полне
 Сторит расстрыли со сегодня
 сам расно спидете.
 
 Тишина в короткий!
 Пойду я трова
 Помогайте друг другу прокушки
 сам воздву помню,
 Покупай Комбат
 
 не получай.
 Но и СПИД сойдет.
 
 Прохладный ветер.
 и с гор, парус или тужу,
 Был в туалете
 
 Пропил сапоги
 теперь и вечен…
 
 Пролетарии
 вернись и жива?
 
 Огород ин.
 На ноге бомжа
 
 проверяя скрипт,
 но не надо света.
 
 Плывут моей милой
 Как японский бог
 
 Как там Воркута?
 Волкому махман я —
 лебединая хлом.
 Блин моя зай